In [ ]:
import exmp
import qiime2
import tempfile
import os.path
import pandas as pd
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import collapse

# EXMP 1 

In [ ]:
taxonomy = exmp.load_taxonomy()
sample_metadata = exmp.load_sample_metadata()
data_dir = exmp.cm_path

rarefied_table = qiime2.Artifact.load(os.path.join(data_dir, "rarefied_table.qza"))
uu_dm = qiime2.Artifact.load(os.path.join(data_dir, "unweighted_unifrac_distance_matrix.qza"))
wu_dm = qiime2.Artifact.load(os.path.join(data_dir, "weighted_unifrac_distance_matrix.qza"))
faith_pd = qiime2.Artifact.load(os.path.join(data_dir, "faith_pd_vector.qza"))
shannon = qiime2.Artifact.load(os.path.join(data_dir, "shannon_vector.qza"))
evenness = qiime2.Artifact.load(os.path.join(data_dir, "evenness_vector.qza"))

In [ ]:
with tempfile.TemporaryDirectory() as output_dir:
    _, _, _, sample_metadata = exmp.ols_and_anova('RER_change', 'exmp1', '1.0', 
                                                  output_dir, 'week',
                                                  sample_metadata, uu_dm, wu_dm, 
                                                  faith_pd, shannon, evenness)

In [ ]:
rarefied_table = filter_samples(table=rarefied_table, metadata=sample_metadata).filtered_table
taxa_table = collapse(table=rarefied_table, taxonomy=taxonomy, level=6).collapsed_table.view(pd.DataFrame)

In [ ]:
sample_metadata = sample_metadata.to_dataframe()

In [ ]:
sorted_wu_pc3_correlations = pd.DataFrame(taxa_table.corrwith(sample_metadata['Weighted_UniFrac_PC3'], method='spearman').sort_values(), columns=['Spearman rho'])
sorted_wu_pc3_correlations['25th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.25)
sorted_wu_pc3_correlations['Median rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.50)
sorted_wu_pc3_correlations['75th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.75)

The data are most easily interpreted if the ordination axes are positively correlated with the RER change. Since the direction of the PCs are arbitrary, I generally just run this a few times till I get a positive correlation.

In [ ]:
sample_metadata['Weighted_UniFrac_PC3'].corr(sample_metadata['RER_change'])

In [ ]:
output_dir = os.path.join(exmp.cm_path, 'ols-and-anova', 'exmp1-RER_change-week1.0')
sorted_wu_pc3_correlations.to_csv(open(os.path.join(output_dir, 'wu-pcoa3-genus-correlations.csv'), 'w'))

# EXMP 2

In [ ]:
taxonomy = exmp.load_taxonomy()
sample_metadata = exmp.load_sample_metadata()
data_dir = exmp.cm_path

rarefied_table = qiime2.Artifact.load(os.path.join(data_dir, "rarefied_table.qza"))
uu_dm = qiime2.Artifact.load(os.path.join(data_dir, "unweighted_unifrac_distance_matrix.qza"))
wu_dm = qiime2.Artifact.load(os.path.join(data_dir, "weighted_unifrac_distance_matrix.qza"))
faith_pd = qiime2.Artifact.load(os.path.join(data_dir, "faith_pd_vector.qza"))
shannon = qiime2.Artifact.load(os.path.join(data_dir, "shannon_vector.qza"))
evenness = qiime2.Artifact.load(os.path.join(data_dir, "evenness_vector.qza"))

In [ ]:
with tempfile.TemporaryDirectory() as output_dir:
    _, _, _, sample_metadata = exmp.ols_and_anova('three_rep_max_squat_change', 'exmp2', '1.0', 
                                                  output_dir, 'week',
                                                  sample_metadata, uu_dm, wu_dm, 
                                                  faith_pd, shannon, evenness)

In [ ]:
rarefied_table = filter_samples(table=rarefied_table, metadata=sample_metadata).filtered_table
taxa_table = collapse(table=rarefied_table, taxonomy=taxonomy, level=6).collapsed_table.view(pd.DataFrame)

In [ ]:
sample_metadata = sample_metadata.to_dataframe()

In [ ]:
sorted_wu_pc2_correlations = pd.DataFrame(taxa_table.corrwith(sample_metadata['Weighted_UniFrac_PC2'], method='spearman').sort_values(), columns=['Spearman rho'])
sorted_wu_pc2_correlations['25th percentile rarefied count'] = taxa_table[sorted_wu_pc2_correlations.index].quantile(0.25)
sorted_wu_pc2_correlations['Median rarefied count'] = taxa_table[sorted_wu_pc2_correlations.index].quantile(0.50)
sorted_wu_pc2_correlations['75th percentile rarefied count'] = taxa_table[sorted_wu_pc2_correlations.index].quantile(0.75)
sorted_wu_pc3_correlations = pd.DataFrame(taxa_table.corrwith(sample_metadata['Weighted_UniFrac_PC3'], method='spearman').sort_values(), columns=['Spearman rho'])
sorted_wu_pc3_correlations['25th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.25)
sorted_wu_pc3_correlations['Median rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.50)
sorted_wu_pc3_correlations['75th percentile rarefied count'] = taxa_table[sorted_wu_pc3_correlations.index].quantile(0.75)

In [ ]:
sample_metadata['Weighted_UniFrac_PC2'].corr(sample_metadata['three_rep_max_squat_change'])

In [ ]:
sample_metadata['Weighted_UniFrac_PC3'].corr(sample_metadata['three_rep_max_squat_change'])

In [ ]:
output_dir = os.path.join(exmp.cm_path, 'ols-and-anova', 'exmp2-three_rep_max_squat_change-week1.0')
sorted_wu_pc2_correlations.to_csv(open(os.path.join(output_dir, 'wu-pcoa2-genus-correlations.csv'), 'w'))
sorted_wu_pc3_correlations.to_csv(open(os.path.join(output_dir, 'wu-pcoa3-genus-correlations.csv'), 'w'))